## Data file:
* https://raw.githubusercontent.com/vjavaly/Baruch-CIS-STA-3920/main/data/Seattle_hotels.csv

In [1]:
from datetime import datetime
print(f'Run time: {datetime.now().strftime("%D %T")}')

Run time: 03/31/24 00:09:20


### Import libraries

In [2]:
import pandas as pd
import re
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\raisa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\raisa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\raisa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load data

In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/vjavaly/Baruch-CIS-STA-3920/main/data/Seattle_hotels.csv")
df.shape

(152, 3)

### Examine data

In [4]:
#sets the maximum column width to unlimited
pd.set_option('max_colwidth', None)

In [5]:
#displays first 5 rows
df.head()

,name,address,desc
0,Hilton Garden Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA","Located on the southern tip of Lake Union, the Hilton Garden Inn Seattle Downtown hotel is perfectly located for business and leisure. \nThe neighborhood is home to numerous major international companies including Amazon, Google and the Bill & Melinda Gates Foundation. A wealth of eclectic restaurants and bars make this area of Seattle one of the most sought out by locals and visitors. Our proximity to Lake Union allows visitors to take in some of the Pacific Northwest's majestic scenery and enjoy outdoor activities like kayaking and sailing. over 2,000 sq. ft. of versatile space and a complimentary business center. State-of-the-art A/V technology and our helpful staff will guarantee your conference, cocktail reception or wedding is a success. Refresh in the sparkling saltwater pool, or energize with the latest equipment in the 24-hour fitness center. Tastefully decorated and flooded with natural light, our guest rooms and suites offer everything you need to relax and stay productive. Unwind in the bar, and enjoy American cuisine for breakfast, lunch and dinner in our restaurant. The 24-hour Pavilion Pantry? stocks a variety of snacks, drinks and sundries."
1,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA","Located in the city's vibrant core, the Sheraton Grand Seattle provides a gateway to the diverse sights and sounds of the Pacific Northwest. Step out of our front doors to find gourmet dining and bars, world-class shopping, exciting entertainment, and iconic local attractions including the Pike Place Market, Space Needle and Chihuly Garden & Glass Museum. As one of only seven Sheraton hotels in North America to earn the esteemed Grand designation, guests can book confidently knowing they?re receiving the highest benchmark on product and service offerings available. Experience our recently completed multimillion-dollar transformation featuring all new guest rooms, an expanded Sheraton Club Lounge, and modern meeting & event spaces. Gather in our stylish new lobby and enjoy our private art collection featuring local artists while enjoying your favorite beverage from Starbucks. The Sheraton Grand features several dining options including Loulay Kitchen & Bar by James Beard award winning chef Thierry Rautureau."
2,Crowne Plaza Seattle Downtown,"1113 6th Ave, Seattle, WA 98101","Located in the heart of downtown Seattle, the award-winning \nCrowne Plaza Hotel Seattle ? Downtown offers an exceptional blend of service, style and comfort. You?ll notice Cool, Comfortable and Unconventional touches that set us apart as soon as you step inside. Marvel at stunning views of the city lights while relaxing in our new Sleep Advantage? Beds. Enjoy complimentary wireless Internet throughout the hotel and amenities to help you relax like our Temple Spa? Sleep Tight Amenity kits featuring lavender spray and lotions to help you rejuvenate and unwind. Enjoy an invigorating workout at our 24-hour fitness center, get dining suggestions from our expert concierge or savor sumptuous cuisine at our Regatta Bar & Grille restaurant where you can enjoy Happy Hour in our lounge daily from 4pm - 7pm and monthly drink specials. Come and experience all that The Emerald City has to offer with us!"
3,Kimpton Hotel Monaco Seattle,"1101 4th Ave, Seattle, WA98101","What?s near our hotel downtown Seattle location? The better \nquestion might be what?s not nearby. In addition to being one of the hotels near Pike Place Market, here?s just a small sampling of the rest. Columbia Center, whose Sky View Observatory on the 73rd floor is the tallest public viewing area west of the Mississippi Historic 5th Avenue Theatre, home to musical productions Seattle Central Library, an architectural marvel. Within half a mile: The must-see Pike Place Market, which houses the original Starbucks Pioneer Square, Seattle?s original downtown. Seattle Art Museum. 

### Prepare data

In [6]:
# Drop column address
df.drop('address', axis=1, inplace=True)

#### Clean column hotel descriptions
1) remove punctuation
2) lowercase text
3) either stem or lemmatize text

In [7]:
#defines a function called 'clean_text' that preprocesses text data
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
def clean_text(text):
    lem = WordNetLemmatizer()
    stop = set(stopwords.words('english'))
    punct = string.punctuation
    text = re.sub(r'\s+', ' ', text)
    text = text.translate(str.maketrans('', '', punct)).lower()
    tokens = re.split(r'\W+', text)
    tokens = [lem.lemmatize(word) for word in tokens if word not in stop]
    return ' '.join(tokens)

df['desc_cleaned'] = df['desc'].apply(clean_text)


#### Display updated dataframe

In [8]:
df.head()

,name,desc,desc_cleaned
0,Hilton Garden Seattle Downtown,"Located on the southern tip of Lake Union, the Hilton Garden Inn Seattle Downtown hotel is perfectly located for business and leisure. \nThe neighborhood is home to numerous major international companies including Amazon, Google and the Bill & Melinda Gates Foundation. A wealth of eclectic restaurants and bars make this area of Seattle one of the most sought out by locals and visitors. Our proximity to Lake Union allows visitors to take in some of the Pacific Northwest's majestic scenery and enjoy outdoor activities like kayaking and sailing. over 2,000 sq. ft. of versatile space and a complimentary business center. State-of-the-art A/V technology and our helpful staff will guarantee your conference, cocktail reception or wedding is a success. Refresh in the sparkling saltwater pool, or energize with the latest equipment in the 24-hour fitness center. Tastefully decorated and flooded with natural light, our guest rooms and suites offer everything you need to relax and stay productive. Unwind in the bar, and enjoy American cuisine for breakfast, lunch and dinner in our restaurant. The 24-hour Pavilion Pantry? stocks a variety of snacks, drinks and sundries.",located southern tip lake union hilton garden inn seattle downtown hotel perfectly located business leisure neighborhood home numerous major international company including amazon google bill melinda gate foundation wealth eclectic restaurant bar make area seattle one sought local visitor proximity lake union allows visitor take pacific northwest majestic scenery enjoy outdoor activity like kayaking sailing 2000 sq ft versatile space complimentary business center stateoftheart av technology helpful staff guarantee conference cocktail reception wedding success refresh sparkling saltwater pool energize latest equipment 24hour fitness center tastefully decorated flooded natural light guest room suite offer everything need relax stay productive unwind bar enjoy american cuisine breakfast lunch dinner restaurant 24hour pavilion pantry stock variety snack drink sundries
1,Sheraton Grand Seattle,"Located in the city's vibrant core, the Sheraton Grand Seattle provides a gateway to the diverse sights and sounds of the Pacific Northwest. Step out of our front doors to find gourmet dining and bars, world-class shopping, exciting entertainment, and iconic local attractions including the Pike Place Market, Space Needle and Chihuly Garden & Glass Museum. As one of only seven Sheraton hotels in North America to earn the esteemed Grand designation, guests can book confidently knowing they?re receiving the highest benchmark on product and service offerings available. Experience our recently completed multimillion-dollar transformation featuring all new guest rooms, an expanded Sheraton Club Lounge, and modern meeting & event spaces. Gather in our stylish new lobby and enjoy our private art collection featuring local artists while enjoying your favorite beverage from Starbucks. The Sheraton Grand features several dining options including Loulay Kitchen & Bar by James Beard award winning chef Thierry Rautureau.",located city vibrant core sheraton grand seattle provides gateway diverse sight sound pacific northwest step front door find gourmet dining bar worldclass shopping exciting entertainment iconic local attraction including pike place market space needle chihuly garden glass museum one seven sheraton hotel north america earn esteemed grand designation guest book confidently knowing theyre receiving highest benchmark product service offering available experience recently completed multimilliondollar transformation featuring new guest room expanded sheraton club lounge modern meeting event space gather stylish new lobby enjoy private art collection featuring local artist enjoying favorite beverage starbucks sheraton grand feature several dining option including loulay kitchen bar james beard award winning chef thierry rautureau
2,Cr

### Vectorize cleaned hotel descriptions

In [9]:
tfidf = TfidfVectorizer(ngram_range=(1, 3))
tfidf_matrix = tfidf.fit_transform(df['desc_cleaned'])
tfidf_matrix.shape

(152, 27459)

In [10]:
type(tfidf_matrix)
print(tfidf_matrix.getrow(0))

  (0, 22463)	0.06071661794144815
  (0, 25757)	0.06071661794144815
  (0, 23541)	0.06071661794144815
  (0, 17742)	0.06071661794144815
  (0, 17945)	0.06071661794144815
  (0, 354)	0.06071661794144815
  (0, 19871)	0.06071661794144815
  (0, 6794)	0.06071661794144815
  (0, 14595)	0.06071661794144815
  (0, 3564)	0.052831427761104735
  (0, 6195)	0.06071661794144815
  (0, 1586)	0.06071661794144815
  (0, 7723)	0.06071661794144815
  (0, 2657)	0.06071661794144815
  (0, 25535)	0.06071661794144815
  (0, 18831)	0.06071661794144815
  (0, 23393)	0.06071661794144815
  (0, 19615)	0.06071661794144815
  (0, 16355)	0.06071661794144815
  (0, 8265)	0.06071661794144815
  (0, 16984)	0.06071661794144815
  (0, 23927)	0.06071661794144815
  (0, 20482)	0.05610407737489257
  (0, 10825)	0.046465280657644194
  (0, 13792)	0.06071661794144815
  :	:
  (0, 15359)	0.050292963508522255
  (0, 3241)	0.050292963508522255
  (0, 10453)	0.046465280657644194
  (0, 1432)	0.03572115644765015
  (0, 12483)	0.027835966267306732
  (0, 545

### Generate similarities matrix on cleaned hotel descriptions

In [11]:
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

### Create hotel recommender

In [12]:
def recommend_hotels(hotel_name, similarity_matrix, hotel_names):
    
    hotel_index = hotel_names[hotel_names == hotel_name].index[0]
    similarity_scores = similarity_matrix[hotel_index]
    similar_hotel_indices = similarity_scores.argsort()[:-4:-1]
    recommended_hotels = hotel_names.iloc[similar_hotel_indices].tolist()
    
    return recommended_hotels


### Make hotel recommendations for the following hotel names:
* Motel 6 Seattle Sea-Tac Airport South
* The Bacon Mansion Bed and Breakfast
* Holiday Inn Seattle Downtown

In [13]:
recommend_hotels("Motel 6 Seattle Sea-Tac Airport South", similarity_matrix, df['name'])

['Motel 6 Seattle Sea-Tac Airport South',
 'Emerald Motel',
 'Ramada by Wyndham SeaTac Airport']

In [14]:
recommend_hotels("The Bacon Mansion Bed and Breakfast", similarity_matrix, df['name'])

['The Bacon Mansion Bed and Breakfast',
 '11th Avenue Inn Bed and Breakfast',
 'Shafer Baillie Mansion Bed & Breakfast']

In [15]:
recommend_hotels("Holiday Inn Seattle Downtown", similarity_matrix, df['name'])

['Holiday Inn Seattle Downtown',
 'Holiday Inn Express & Suites Seattle-City Center',
 'Inn at Queen Anne']